# 제품 이상여부 판별 프로젝트


### 데이터 읽어오기


In [10]:
import pandas as pd

RANDOM_STATE = 110

train_data = pd.read_csv("../../../data/train_data_0816.csv")
test_data = pd.read_csv("../../../data/test_data_0816.csv")

In [11]:
# 공통 변수 리스트
com_variables_train = [
    'target', 'Model.Suffix', 'Workorder', 'WorkMode Collect Result'
    , 'Dispenser_1', 'Dispenser_2', 'Receip_No_Collect_Result'
    , 'Production_Qty_Collect_Result', 'Judge_Value_OK'
    , 'Workorder_0.9', 'Workorder_0.6'
]

com_variables_test = [
    'target', 'Set ID', 'Model.Suffix', 'Workorder'
    , 'WorkMode Collect Result', 'Dispenser_1'
    , 'Dispenser_2', 'Receip_No_Collect_Result'
    , 'Production_Qty_Collect_Result', 'Judge_Value_OK'
    , 'Workorder_0.9', 'Workorder_0.6'
]

In [12]:
# 공정 이름 필터링 후 공통 변수와 결합
def create_dataset(train_data, test_data, process_name, com_variables_train, com_variables_test):
    # 열 이름 필터링
    Process_Desc_col = train_data.filter(like=process_name).columns
    
    # train 데이터셋 생성
    final_columns_train = list(Process_Desc_col) + com_variables_train
    train_dataset = train_data[final_columns_train]
    
    # test 데이터셋 생성
    final_columns_test = list(Process_Desc_col) + com_variables_test
    test_dataset = test_data[final_columns_test]
    
    return train_dataset, test_dataset

# 공통 변수 정의
## com_variables_train = [...]  -> 이전 코드에서 정의한 변수 사용
## com_variables_test = [...]   -> 이전 코드에서 정의한 변수 사용

# 데이터셋 생성
train_data_dam, test_data_dam = create_dataset(train_data, test_data, '_Dam', com_variables_train, com_variables_test)
train_data_fill1, test_data_fill1 = create_dataset(train_data, test_data, '_Fill1', com_variables_train, com_variables_test)
train_data_fill2, test_data_fill2 = create_dataset(train_data, test_data, '_Fill2', com_variables_train, com_variables_test)
train_data_autoclave, test_data_autoclave = create_dataset(train_data, test_data, '_AutoClave', com_variables_train, com_variables_test)

---

## Optuna

스레스홀드 0.3으로 맞춘상태에서 튜닝 진행한 것

In [13]:
# import optuna
# from xgboost import XGBClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import f1_score

# # 'Normal'과 'AbNormal'을 숫자로 변환
# train_data_fill1['target'] = train_data_fill1['target'].map({'Normal': 0, 'AbNormal': 1})

# # 스레드홀드 설정
# THRESHOLD = 0.3

# def objectiveXGB(trial, x_tr, y_tr, x_val, y_val):
#     param = {
#         'n_estimators': trial.suggest_int('n_estimators', 300, 2000)
#         , 'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.5)
#         , 'max_depth': trial.suggest_int('max_depth', 3, 30)

#         , 'alpha': trial.suggest_float('alpha', 0.0001, 0.1, log=True)
#         , 'gamma': trial.suggest_float('gamma', 0.0001, 0.1, log=True)

#         , 'reg_alpha' : trial.suggest_float('reg_alpha', 0.01, 1)
#         , 'reg_lambda' : trial.suggest_float('reg_lambda', 0.01, 1)
        
#         , 'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.01, 1)
#         , 'subsample' : trial.suggest_float('subsample', 0.01, 1)
#         , 'objective': 'binary:logistic'  # 이진 분류
#         , 'tree_method' : "exact"        
#         , 'random_state': RANDOM_STATE
#     }
       
#     model = XGBClassifier(**param)
#     model.fit(x_tr, y_tr)
#     pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
#     pred = (pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측
    
#     score = f1_score(y_val, pred, average="binary")
    
#     return score

# # 데이터셋 분할
# x_train, x_val, y_train, y_val = train_test_split(
#     train_data_fill1.drop("target", axis=1),
#     train_data_fill1["target"],
#     test_size=0.2,
#     shuffle=True,
#     random_state=RANDOM_STATE,
# )

# # 하이퍼 파라미터 튜닝
# study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE))
# study.optimize(lambda trial: objectiveXGB(trial, x_train, y_train, x_val, y_val), n_trials=200)

# print('Best trial: score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))


Best trial: score 0.23052959501557632,  
params {'n_estimators': 1623, 'learning_rate': 0.2598278142183139, 'max_depth': 3, 'alpha': 0.06616394656827354,  
'gamma': 0.003347718643022639, 'reg_alpha': 0.9185363939667319, 'reg_lambda': 0.2387201886189512,  
'colsample_bytree': 0.9006465039205399, 'subsample': 0.7359523800973177}

### 두번째 optuna 진행

In [14]:
import optuna
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# 'Normal'과 'AbNormal'을 숫자로 변환
train_data_fill1['target'] = train_data_fill1['target'].map({'Normal': 0, 'AbNormal': 1})

# 스레드홀드 설정
THRESHOLD = 0.3

def objectiveXGB(trial, x_tr, y_tr, x_val, y_val):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 1000, 2500)
        , 'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2)
        , 'max_depth': trial.suggest_int('max_depth', 3, 10)

        , 'alpha': trial.suggest_float('alpha', 0.0001, 0.01, log=True)
        , 'gamma': trial.suggest_float('gamma', 0.0001, 0.01, log=True)

        , 'reg_alpha' : trial.suggest_float('reg_alpha', 0.01, 1)
        , 'reg_lambda' : trial.suggest_float('reg_lambda', 0.01, 1)
        
        , 'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.01, 1)
        , 'subsample' : trial.suggest_float('subsample', 0.01, 1)
        , 'objective': 'binary:logistic'  # 이진 분류
        , 'tree_method' : "exact"        
        , 'random_state': RANDOM_STATE
    }
       
    model = XGBClassifier(**param)
    model.fit(x_tr, y_tr)
    pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
    pred = (pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측
    
    score = f1_score(y_val, pred, average="binary")
    
    return score

# 데이터셋 분할
x_train, x_val, y_train, y_val = train_test_split(
    train_data_fill1.drop("target", axis=1),
    train_data_fill1["target"],
    test_size=0.2,
    shuffle=True,
    random_state=RANDOM_STATE,
)

# 하이퍼 파라미터 튜닝
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE))
study.optimize(lambda trial: objectiveXGB(trial, x_train, y_train, x_val, y_val), n_trials=150)

print('Best trial: score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))


C:\Users\juneh\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
[I 2024-08-17 10:29:57,195] A new study created in memory with name: no-name-0e123ce2-7eaa-4106-a4ef-44d880474670
[I 2024-08-17 10:30:07,372] Trial 0 finished with value: 0.21265141318977118 and parameters: {'n_estimators': 1174, 'learning_rate': 0.13510652253176916, 'max_depth': 6, 'alpha': 0.001680990128610233, 'gamma': 0.002285499365417053, 'reg_alpha': 0.685638497930364, 'reg_lambda': 0.9554598960277283, 'colsample_bytree': 0.640850709997791, 'subsample': 0.5178141716690416}. Best is trial 0 with value: 0.21265141318977118.
[I 2024-08-17 10:30:12,699] Trial 1 finished with value: 0.1724137931034483 and

Best trial: score 0.23407022106631986, 
params {'n_estimators': 2213, 'learning_rate': 0.11735028445102921, 'max_depth': 5, 'alpha': 0.008053553080773119, 'gamma': 0.0006061064433044652, 'reg_alpha': 0.5380410453451789, 'reg_lambda': 0.8278020434040504, 'colsample_bytree': 0.8672296781322193, 'subsample': 0.3694461716611997}


Best trial: score 0.23407022106631986,  
params {'n_estimators': 2213, 'learning_rate': 0.11735028445102921, 'max_depth': 5, 'alpha': 0.008053553080773119,  
'gamma': 0.0006061064433044652, 'reg_alpha': 0.5380410453451789, 'reg_lambda': 0.8278020434040504,  
'colsample_bytree': 0.8672296781322193, 'subsample': 0.3694461716611997}

.